## 네이버 View 검색을 활용한 네이버 블로그 크롤링

In [1]:
from selenium import webdriver
import time
from bs4 import BeautifulSoup as bs
import csv 
import requests
import pandas as pd
import re
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.common.exceptions import UnexpectedAlertPresentException
import urllib.request
import random
import warnings 
warnings.filterwarnings(action='ignore') 

#### naver view 기본 구조

- 스크롤 1번에 30개 url 로드

- 스크롤 3번 내리면 블로그 주소 총 120개 생성

- VIEW 페이지 블로그 url html 클래스 : api_txt_lines total_tit
  
#### 최대 수집 가능한 url 개수

- 1000개

    확인 결과 네이버 VIEW에서는 최대 1000개의 블로그 url이 로드됩니다.
    그 이상 스크롤을 내려도 (즉,1000개를 초과해서도) 로딩이 됩니다면 마지막 30개 url이 반복해서 등장할 뿐입니다.
    
    따라서 하단에 원하는 블로그 개수 input을 1020으로 입력하시면 해당 keyword에 대한 max값을 입력했다고 볼 수 있겠습니다. (그 이상으로 해봤자 동일한 url이 반복돼서 append될 뿐입니다)
    
    더 많은 텍스트 데이터 수집을 위한 대안으로 keyword(검색어)를 달리해서 원하시는 토픽에 대한 블로그 url을 수집하시길 바랍니다.
    또한 그 과정에 중복되는 url이 있을수도 있으니 url을 담은 리스트를 set type으로 전환해서 중복된 값을 제거한 후 크롤링을 진행하시면 동일한 블로그를 두번 수집하는 경우를 방지할 수 있습니다.

In [2]:
#수집하고 싶은 기간
first_days = pd.date_range('2022/01/01', '2022/03/31', freq='MS') #MS는 해당 월 초 날짜 산출
last_days = pd.date_range('2022/01/01', '2022/03/31', freq='M') #M은 해당 월 말 날짜 산출
first_days

DatetimeIndex(['2022-01-01', '2022-02-01', '2022-03-01'], dtype='datetime64[ns]', freq='MS')

In [3]:
#첫번째 input으로 검색 키워드를 설정합니다 (e.g., "자동차")
keyword = input('키워드를 입력해주세요 :')

#url을 설정합니다
url = 'https://search.naver.com/search.naver?sm=tab_hty.top&where=blog&query={}&oquery={}'.format(keyword, keyword)

#크롬 드라이버 선언
driver = webdriver.Chrome("chromedriver.exe") #현재 컴퓨터의 크롬드라이버 위치 지정
driver.implicitly_wait(1)

#네이버 VIEW 블로그 검색창 이동
driver.get(url)
time.sleep(1)

#원하는 블로그 개수를 30 단위로 입력 (스크롤 당 30개의 블로그가 업로드 됩니다)
url_want = input('30단위로 원하는 블로그 개수 입력 :')
url_want = int(url_want)

#30의 배수로 숫자를 입력하지 않은 경우 재입력을 받는 코드입니다
while url_want%30 !=0: 
    url_want = int(input('30단위로 원하는 블로그 개수 입력 :'))
    if url_want%30 ==0:
         break

키워드를 입력해주세요 :차박일기
30단위로 원하는 블로그 개수 입력 :60


In [4]:
blog_url_list=[]

# 검색 옵션 클릭하기
driver.find_element_by_xpath('//*[@id="snb"]/div[1]/div/div[2]/a').click()
for k in range(len(first_days)):
    # 직접입력 클릭하기
    driver.find_element_by_xpath('//*[@id="snb"]/div[2]/ul/li[3]/div/div[1]/a[9]/i').click()
    time.sleep(0.8)    

    #시작 날짜 지정#년(year) 클릭 
    driver.find_element_by_xpath('//*[@id="snb"]/div[2]/ul/li[3]/div/div[2]/div[2]/div[1]/div/div/div/ul/li[{0}]'.format(first_days.year[k]-2002)).click()
    #월(momth) 클릭
    driver.find_element_by_xpath('//*[@id="snb"]/div[2]/ul/li[3]/div/div[2]/div[2]/div[2]/div/div/div/ul/li[{0}]/a'.format(first_days.month[k])).click()
    #일 (day) 클릭
    driver.find_element_by_xpath('//*[@id="snb"]/div[2]/ul/li[3]/div/div[2]/div[2]/div[3]/div/div/div/ul/li[{0}]/a'.format(first_days.day[k])).click()

    #끝 날짜 클릭
    driver.find_element_by_xpath('//*[@id="snb"]/div[2]/ul/li[3]/div/div[2]/div[1]/span[3]/a').click()
    #년(year) 클릭 
    driver.find_element_by_xpath('//*[@id="snb"]/div[2]/ul/li[3]/div/div[2]/div[2]/div[1]/div/div/div/ul/li[{0}]'.format(last_days.year[k]-2002)).click()
    #월(momth) 클릭
    driver.find_element_by_xpath('//*[@id="snb"]/div[2]/ul/li[3]/div/div[2]/div[2]/div[2]/div/div/div/ul/li[{0}]/a'.format(last_days.month[k])).click()
    #일 (day) 클릭
    driver.find_element_by_xpath('//*[@id="snb"]/div[2]/ul/li[3]/div/div[2]/div[2]/div[3]/div/div/div/ul/li[{0}]/a'.format(last_days.day[k])).click()
    
    #적용 버튼 클릭하기
    driver.find_element_by_xpath('//*[@id="snb"]/div[2]/ul/li[3]/div/div[2]/div[3]/button').click()
    # 스크롤 위치 인식
    last_height = driver.execute_script("return document.body.scrollHeight") 

    #스크롤 횟수 (1회 스크롤당 게시물 30개 보입니다)
    scroll = (url_want/30)-1 
        
    # 지정한 만큼 스크롤 다운
    for i in range(int(scroll)):                                                                  
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);") #스크롤을 가능한 아래까지 내립니다.
        time.sleep(random.uniform(1,1.7))
     
        new_height = driver.execute_script("return document.body.scrollHeight")  #스크롤을 가능한 아래까지 내렸을 때, 갱신되어 스크롤의 위치와 크기가 바뀌면, 다시 그 스크롤을 최대한 아래까지 내립니다.
        if new_height == last_height:   
            break
                                                                             
        last_height = new_height               
                                                                             
    # 블로그 url 긁어오기
    soup = bs(driver.page_source, 'lxml') 
    blog_url = soup.find_all('a',class_='api_txt_lines total_tit')
        
    # url 추출, 리스트에 정리
    for i in blog_url:
        blog_url_list.append(i['href'])
            
    print('{0}년{1}월'.format(first_days.year[k], first_days.month[k]), len(blog_url),' 개 블로그 url주소 수집')
        
    #스크롤 맨위로    
    driver.execute_script("window.scrollTo(0, 0)")
    time.sleep(0.8)

2022년1월 60  개 블로그 url주소 수집
2022년2월 60  개 블로그 url주소 수집
2022년3월 60  개 블로그 url주소 수집


#### 요청한만큼 스크롤이 내려간 VIEW 페이지에서 url을 수집

수집 후 blog_url_list를 set type으로 전환해 혹시 중복된 url이 있는지 확인하시면 좋을 것 같습니다.

In [5]:
len(blog_url_list)

180

In [6]:
real_list = set(blog_url_list)
blog_url_list = list(real_list)
len(blog_url_list)

180

In [7]:
k = pd.DataFrame(blog_url_list)
k.to_csv('blog_url_list.csv')

#### 수집한 블로그 url로 이동하여 본문 크롤링

- 프레임 전환 driver.switch_to.frame('mainFrame') 


In [8]:
url = pd.read_csv('blog_url_list.csv')
blog_url_list = list(url['0'])
blog_url_list[:5]

['https://blog.naver.com/star4621?Redirect=Log&logNo=222677246975',
 'https://blog.daum.net/boodjja71/1484',
 'https://blog.naver.com/for_sbe?Redirect=Log&logNo=222610182909',
 'https://blog.daum.net/boodjja71/1487',
 'https://blog.naver.com/realty0179?Redirect=Log&logNo=222612789513']

In [9]:
no_naver_blog1=[] #url 수집중 naver 블로그가 아닌 경우 제외시키기 위한 리스트

start = time.time() # 시작 시간 체크

#본문, 댓글, 날짜, url 빈 list 생성
review_list = []
comment_list = []
like_list = []
time_list = []
url_list = []
count = 0

driver = webdriver.Chrome("chromedriver.exe")

for url in blog_url_list[:20]: #20개로 지정해놓음**
    #driver = webdriver.Chrome("chromedriver.exe")
    if 'naver' in url: #url에 naver가 들어있는 경우에만 진행
        count += 1
        driver.get(url)
        time.sleep(1.1)

        #프레임 전환
        try: 
            driver.switch_to.frame('mainFrame') 
            time.sleep(1)
        
            # 블로그의 내용 크롤링
            soup = bs(driver.page_source, 'lxml') 
            postview = soup.find('div', attrs={'id': re.compile('post-view.+')}).get_text()

            reg = re.compile(r'[\s+]')
            postview_reg = reg.sub(' ',postview)
            
            #공감 수 
            try: 
                like = driver.find_element_by_xpath("//span[@class='u_likeit_list_btn _button btn_sympathy pcol2 off']/em[@class='u_cnt _count']")
                like = like.text
            except:
                like = []
            
            #작성 시간
            try: 
                timeline = driver.find_element_by_xpath("//span[@class='se_publishDate pcol2']")
            except:
                timeline = driver.find_element_by_xpath("//p[@class='date fil5 pcol2 _postAddDate']")
            timeline = timeline.text

            #댓글
            try:
                # 댓글 버튼 클릭
                driver.find_element_by_css_selector('span.btn_arr').click()
                time.sleep(2)

                #댓글 수집 (대댓글도 같이 수집) (비밀댓글은 수집 안됩니다)
                comment_blog = []
                comment = driver.find_elements_by_css_selector('span.u_cbox_contents')
                for i in comment:
                    com = str(i.text)
                    com = reg.sub(' ',com)
                    comment_blog.append(([com]))
            except: 
                # 댓글 버튼이 없는 블로그의 경우 패스
                comment_blog = []
             
            
            #수집 내용을 빈리스트에 append
            review_list.append(postview_reg)
            like_list.append(like)
            time_list.append(timeline)
            comment_list.append(comment_blog)
            url_list.append(url)
            time.sleep(random.uniform(1,1.7)) #1초에서 1.7초 사이 랜덤으로 쉬어가기
            
        except UnexpectedAlertPresentException:
            pass
        
    else: #naver블로그가 아닌 경우
        no_naver_blog1.append(url)
        print('네이버 블로그 아닌 url: ',url)
    
    if count%10 ==0: #진행상황 출력
        print('{}개 블로그 크롤링 완료'.format(count))

네이버 블로그 아닌 url:  https://blog.daum.net/boodjja71/1484
네이버 블로그 아닌 url:  https://blog.daum.net/boodjja71/1487
10개 블로그 크롤링 완료


In [11]:
# 수집한 리스트를 DataFrame으로 전환
data = {'review' : review_list,
        'comment' : comment_list,
        'like' : like_list,
        'time' : time_list,
        'url': url_list}
df = pd.DataFrame(data)

# 총 걸린시간 출력
print('걸린 시간: ', time.time()-start)
print('최종 {}개 블로그 크롤링 완료'.format(count))
df

걸린 시간:  201.60512614250183
최종 18개 블로그 크롤링 완료


,review,comment,like,time,url
0,용가리 노지캠핑 전남 여수 1박 2일 섬 여행은 낭만적인...,[],17,2022. 3. 19. 17:17,https://blog.naver.com/star4621?Redirect=Log&l...
1,"한량이의 일상 [일상] 한량일기, 연말이야기, 안녕 2021년...",[],7,2022. 1. 1. 13:23,https://blog.naver.com/for_sbe?Redirect=Log&lo...
2,여행 차박(차크닉)하기 좋은 부산의 인생샷 맛집 꿈...,[[여행이간절해요증말]],12,2022. 1. 4. 22:00,https://blog.naver.com/realty0179?Redirect=Log...
3,미라클모닝/감사/건강 [감사일기] 울진에서 일출 (220220...,[[잠자리는 불편하셨지만 가족들과 소중한 추억 남기셨네요~ 너무 편안한 여행보다는 ...,7,2022. 2. 21. 0:15,https://blog.naver.com/jtj_luv?Redirect=Log&lo...
4,일상 행복한 1월 일기 엘리 ・ 2022....,[],18,2022. 1. 27. 14:32,https://blog.naver.com/aily1336?Redirect=Log&l...
5,일상 1월이 가기 전에 쓰는 1월 일기 미동프 ...,[[맛난거 많이 드셨네용! 이삐커피 저번 포스팅 때도 봤는데 가성비 정말 좋은 것 ...,15,2022. 1. 31. 19:25,https://blog.naver.com/alwjd4199?Redirect=Log&...
6,[안전캠페인] 차박캠핑 이것만 알고 가세요!올 겨울! 캠핑이 다시 유행하고 있는 ...,"[[춥지만 차박하러 가고 싶네요.], [차박 캠핑시 꿀팁 감사드립니다!]]",27,2022. 1. 16. 20:25,https://blog.naver.com/number1_max?Redirect=Lo...
7,여행일기 포항 여행추천코스/ 차박하기 좋은 도구해수욕장/ 환호...,[[날풀리면 또 놀자 ㅠㅠ 햄벅해〰️〰️🖤]],8,2022. 2. 5. 16:11,https://blog.naver.com/zzangaaa1213?Redirect=L...
8,일상 ~2월 10일 일기😏 쮜인 ・ 2022....,[],7,2022. 2. 11. 0:13,https://blog.naver.com/djwls338?Redirect=Log&l...
9,"캠핑, 여행 악천후 속 옥천 심천면 차박, 노지 캠핑 성...","[[날씨가 험난했는데도 그래도 마무리는 좋았네요 ㅎㅎ], [감사합니다 즐거운 하루 ...",[],2022. 3. 2. 8:48,https://blog.naver.com/kjc8021?Redirect=Log&lo...


In [12]:
#csv 저장
df.to_csv("blog_view_data.csv", encoding = "utf-8-sig")